# Dependencies / Imports

In [1]:
import pandas as pd
import numpy as np

# DataFrame creation

In [64]:
df = pd.read_csv("C:/Users/rerr_/OneDrive/Desktop/portafolio/data_science_eda/data/dirty_cafe_sales.csv")

# Exploring data

In [12]:
# DataFrame shape (rows, columns)
df.shape

(10000, 8)

In [13]:
# Display the first few rows of the DataFrame
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [14]:
# DataFrame summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [15]:
# Column names
df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

In [16]:
# Data types of each column
df.dtypes

Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

In [17]:
# DataFrame statistics
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_9226047,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [ ]:
# Check for null values
nulls = df.isna().sum()
nulls

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

# Dealing with missing values

## "Item" column

In [80]:
# Count values
df["Item"].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
Unknown      969
Name: count, dtype: int64

In [76]:
# Replace "UNKNOWN" and "ERROR" values with "Unknown"
df["Item"] = df["Item"].replace({"UNKNOWN": "Unknown", 
                    "ERROR": "Unknown"
                    })

df["Item"].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
Unknown      636
Name: count, dtype: int64

In [82]:
# Count the number of null values in the "Item" column
item_null_values = df["Item"].isna().sum()
print(f"There's {item_null_values} null values in the \"Item\" column.")

# Fill null values in the "Item" column with "Unknown"
df["Item"] = df["Item"].fillna("Unknown")

# Check again for null values
post_filtering_item_null_values = df["Item"].isna().sum()
print(f"There's {post_filtering_item_null_values} null values in the \"Item\" column.")

There's 333 null values in the "Item" column.
There's 0 null values in the "Item" column.


In [81]:
df = pd.read_csv("C:/Users/rerr_/OneDrive/Desktop/portafolio/data_science_eda/data/dirty_cafe_sales.csv")

## "Quantity" column

In [83]:
# Count values
df["Quantity"].value_counts()

Quantity
5          2013
2          1974
4          1863
3          1849
1          1822
UNKNOWN     171
ERROR       170
Name: count, dtype: int64

In [84]:
quantity_mean = df["Quantity"].mean()
quantity_median = df["Quantity"].median()
print(f"Quantity mean: {quantity_mean}")
print(f"Quantity median: {quantity_median}")

TypeError: can only concatenate str (not "int") to str

In [35]:
df["Quantity"].value_counts()

Quantity
5          2013
2          1974
4          1863
3          1849
1          1822
UNKNOWN     171
ERROR       170
Name: count, dtype: int64